## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [5]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)


2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
#tf.executing_eagerly()

### Collect Data

In [7]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('/gdrive/My Drive/Great_Lakes_Lab/neural_network/prices.csv')

### Check all columns in the dataset

In [10]:
data.shape

(851264, 7)

In [11]:
data.isna().sum()

date      0
symbol    0
open      0
close     0
low       0
high      0
volume    0
dtype: int64

In [12]:
data.isnull().sum()

date      0
symbol    0
open      0
close     0
low       0
high      0
volume    0
dtype: int64

In [13]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'], axis = 1, inplace= True)

In [15]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data1=data.iloc[0:1000,]

In [17]:
data1.shape

(1000, 5)

In [18]:
data1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [19]:
data1['volume']= data1['volume']/1000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
data1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
data2=data1.values

In [22]:
type(data2)

numpy.ndarray

In [23]:
data2




array([[123.43    , 125.839996, 122.309998, 126.25    ,   2.1636  ],
       [125.239998, 119.980003, 119.940002, 125.540001,   2.3864  ],
       [116.379997, 114.949997, 114.93    , 119.739998,   2.4895  ],
       ...,
       [ 28.32    ,  28.77    ,  28.01    ,  28.809999,  37.1528  ],
       [ 44.      ,  44.799999,  43.75    ,  44.810001,   6.5686  ],
       [ 36.080002,  37.139999,  36.009998,  37.23    ,   5.6043  ]])

In [0]:
from sklearn.model_selection import train_test_split
X = data2[:,0:4]
Y = data2[:,4]
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [25]:
train_x.dtype


dtype('float64')

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

In [28]:
train_y.shape

(700,)

In [0]:
train_y=train_y.reshape(-1,1)

In [30]:
train_y.shape

(700, 1)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.Variable( tf.random.normal(shape=[4,3]))
b1 = tf.zeros(shape=(3))

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.Variable( tf.random.normal(shape=[3,1]))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    #out=tf.sigmoid(net2)
    return net2
  


3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2 = prediction(train_x, w1, b1,w2,b2)
        current_loss = loss(net2,train_y)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [36]:
for i in range(100):
    
    w1, b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)
    print("Loss at step {:d}: {:.3f}".format(i, loss(prediction(train_x, w1, b1,w2,b2), train_y)))

Loss at step 0: 244.554
Loss at step 1: 242.552
Loss at step 2: 240.686
Loss at step 3: 238.949
Loss at step 4: 237.333
Loss at step 5: 235.830
Loss at step 6: 234.436
Loss at step 7: 233.143
Loss at step 8: 231.947
Loss at step 9: 230.841
Loss at step 10: 229.821
Loss at step 11: 228.879
Loss at step 12: 228.013
Loss at step 13: 227.215
Loss at step 14: 226.482
Loss at step 15: 225.809
Loss at step 16: 225.192
Loss at step 17: 224.625
Loss at step 18: 224.106
Loss at step 19: 223.631
Loss at step 20: 223.196
Loss at step 21: 222.797
Loss at step 22: 222.433
Loss at step 23: 222.100
Loss at step 24: 221.796
Loss at step 25: 221.518
Loss at step 26: 221.264
Loss at step 27: 221.033
Loss at step 28: 220.821
Loss at step 29: 220.628
Loss at step 30: 220.452
Loss at step 31: 220.292
Loss at step 32: 220.145
Loss at step 33: 220.012
Loss at step 34: 219.890
Loss at step 35: 219.780
Loss at step 36: 219.679
Loss at step 37: 219.587
Loss at step 38: 219.503
Loss at step 39: 219.427
Loss at st

### Get the shapes and values of W and b

In [37]:
w1.shape

TensorShape([4, 3])

In [38]:
w1


<tf.Tensor: id=6421, shape=(4, 3), dtype=float32, numpy=
array([[ 2.3956203 , -0.16443479, -0.36799926],
       [-1.2411466 , -0.81971556, -1.4760257 ],
       [-0.2829254 ,  0.5074373 ,  1.5546249 ],
       [ 1.2994316 , -1.4742509 ,  0.33966005]], dtype=float32)>

In [39]:
b1.shape

TensorShape([3])

In [40]:
b1


<tf.Tensor: id=6424, shape=(3,), dtype=float32, numpy=array([ 0.9509295 , -0.15769072,  0.05026772], dtype=float32)>

In [41]:
w2.shape

TensorShape([3, 1])

In [42]:
w2

<tf.Tensor: id=6415, shape=(3, 1), dtype=float32, numpy=
array([[ 3.222431  ],
       [-0.02467737],
       [ 0.6687258 ]], dtype=float32)>

In [43]:
b2.shape

TensorShape([1])

In [44]:
b2

<tf.Tensor: id=6418, shape=(1,), dtype=float32, numpy=array([2.4100275], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [45]:
test_data=test_x[0:1,:]
test_data

array([[0.49863145, 0.49986658, 0.49250004, 0.50886524]], dtype=float32)

In [46]:
pred=prediction(test_data, w1, b1,w2,b2)
print("predicted value for 1 example is = ",pred.numpy())

predicted value for 1 example is =  [[5.6036367]]


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data_iris = pd.read_csv('/gdrive/My Drive/Great_Lakes_Lab/neural_network/11_Iris.csv')

In [48]:
data_iris.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
data_iris.drop('Id',axis = 1,inplace=True)

In [50]:
data_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X = data_iris.drop('Species', axis = 1)
Y = data_iris['Species']

In [52]:
X.shape

(150, 4)

In [53]:
Y.shape

(150,)

In [54]:
Y=np.array(Y).reshape(-1,1)
Y = pd.DataFrame(Y)
Y.shape

(150, 1)

In [0]:
Y['species'] = Y

In [0]:
Y = Y.drop(0, axis = 1)

In [57]:
Y.head()

,species
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa


In [58]:
Y =pd.get_dummies(Y)
Y.head()

,species_Iris-setosa,species_Iris-versicolor,species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

### Model Training 

In [61]:
print(tf.__version__)

2.0.0


In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(4,input_dim = 4,activation='sigmoid'))
model.add(tf.keras.layers.Dense(3,activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


### Model Prediction

In [63]:
output = model.fit(train_x, train_y,validation_data = (test_x,test_y), epochs = 50)
print(output)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 1s 8ms/sample - loss: 1.1856 - accuracy: 0.3333 - val_loss: 1.1707 - val_accuracy: 0.3333
Epoch 2/50
105/105 [==============================] - 0s 297us/sample - loss: 1.1798 - accuracy: 0.3333 - val_loss: 1.1657 - val_accuracy: 0.3333
Epoch 3/50
105/105 [==============================] - 0s 259us/sample - loss: 1.1740 - accuracy: 0.3333 - val_loss: 1.1608 - val_accuracy: 0.3333
Epoch 4/50
105/105 [==============================] - 0s 285us/sample - loss: 1.1689 - accuracy: 0.3333 - val_loss: 1.1576 - val_accuracy: 0.3333
Epoch 5/50
105/105 [==============================] - 0s 292us/sample - loss: 1.1655 - accuracy: 0.3333 - val_loss: 1.1535 - val_accuracy: 0.3333
Epoch 6/50
105/105 [==============================] - 0s 368us/sample - loss: 1.1615 - accuracy: 0.3333 - val_loss: 1.1499 - val_accuracy: 0.3333
Epoch 7/50
105/105 [==============================] - 0s 321us/sample - loss: 1.1

### Save the Model

In [0]:
model.save('iris_model.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?